# Export musescore files

Quick script to export musescore files to musicxml and pdf

In [1]:
import glob
import os
import subprocess
import yaml
import music21

import sys
sys.path.append('../')
from hexachord.solmization import Solmization

In [2]:
CORPUS_DIR = '/Users/bas/surfdrive/Research/Guidonian Hand/Corpus'
MSCORE = '/Applications/MuseScore 4.app/Contents/MacOS/mscore'
MdB_DIR = f'{CORPUS_DIR}/Marot & de Bèze'

## MuseScore to musicxml and pdf

In [3]:
def export_source(source_dir, target_ext='musicxml', refresh=False):
    files = sorted(glob.glob(f'{source_dir}/musescore/*.mscz'))
    if not os.path.exists(f'{source_dir}/{target_ext}'):
        os.makedirs(f'{source_dir}/{target_ext}')
    for file in files:
        name, _ =  os.path.basename(file).split('.')
        target = f'{source_dir}/{target_ext}/{name}.{target_ext}'
        
        if name.endswith('-draft'):
            continue

        if not os.path.exists(target) or refresh:
            result = subprocess.run([MSCORE, '-o', target, file])
            print(result)

In [8]:
sources = glob.glob(f'{CORPUS_DIR}/**')
refresh = False
for source_dir in sources:
    export_source(source_dir, target_ext='musicxml', refresh=refresh)
    export_source(source_dir, target_ext='pdf', refresh=refresh)

CompletedProcess(args=['/Applications/MuseScore 4.app/Contents/MacOS/mscore', '-o', '/Users/bas/surfdrive/Research/Guidonian Hand/Corpus/Marot & de Bèze/musicxml/MdB007.musicxml', '/Users/bas/surfdrive/Research/Guidonian Hand/Corpus/Marot & de Bèze/musescore/MdB007.mscz'], returncode=0)
CompletedProcess(args=['/Applications/MuseScore 4.app/Contents/MacOS/mscore', '-o', '/Users/bas/surfdrive/Research/Guidonian Hand/Corpus/Marot & de Bèze/pdf/MdB007.pdf', '/Users/bas/surfdrive/Research/Guidonian Hand/Corpus/Marot & de Bèze/musescore/MdB007.mscz'], returncode=0)


## Evaluate MdB

In [4]:
MdB_META = yaml.safe_load(open(f'{MdB_DIR}/collection.yaml', 'r'))

def process_MdB_work(source_fn, number='?'):
    score = music21.converter.parse(source_fn)
    solmization = Solmization(score)
    solmization.annotate(score.parts[0],
        target_lyrics=2, 
        lyric_offset=3,
        grey_lyrics_num=1
    )
    score.metadata = music21.metadata.Metadata(
        title=f'Psalm {number} (evaluation)',
        composer=MdB_META['composer'],
        lyricist='Automatic solmization: version 30 May 2024'
    )
    return score

# score = process_MdB_work(f"{MdB_DIR}/musicxml/MdB010.musicxml", 1)

In [10]:
refresh = False
target_dir = f'{MdB_DIR}/evaluation'
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

for work in MdB_META['works']:
    id = work['id']
    source_fn = f"{MdB_DIR}/musicxml/{id}.musicxml"
    target_fn = f'{target_dir}/{id}.pdf'
    if not os.path.exists(target_fn) or refresh:
        try:
            score = process_MdB_work(source_fn, number=id)
            score.write('musicxml.pdf', target_fn)
        except Exception as e:
            print('Skipped', source_fn)
            print(e)   

### **TO DO** Other collections

In [6]:
target_dir = 'evaluation'
refresh = True

sources = glob.glob(f'{CORPUS_DIR}/**')
for source_dir in sources[2:3]:
    files = sorted(glob.glob(f'{source_dir}/musicxml/*.musicxml'))
    if not os.path.exists(f'{source_dir}/{target_dir}'):
        os.makedirs(f'{source_dir}/{target_dir}')
    for file in files:
        name, _ =  os.path.basename(file).split('.')
        target_fn = f'{source_dir}/{target_dir}/{name}.pdf'
        if not os.path.exists(target_fn) or refresh:
            try:
                score = converter.parse(file)
                evaluateScore(score, onlyBest=False)
                score.write('musicxml.pdf', target_fn)
            except Exception as e:
                print('Skipped', file)
        

Error annotating note <music21.note.Note G>
Error annotating note <music21.note.Note F>
Error annotating note <music21.note.Note A>
Error annotating note <music21.note.Note E>
